***
# Fundamentals of Data Analytics Assessment - CAO Points Analysis

***

[2021 CAO points]('http://www2.cao.ie/points/l8.php')

[Regular Expression Documentation]('https://docs.python.org/3/library/re.html')

In [23]:
# Regular expressions package
import re 
# Package for HTTP requests.
import requests as rq

In [24]:
# Getting CAO points URL.
resp = rq.get('http://www2.cao.ie/points/l8.php')
# 200 = ok
resp

<Response [200]>

In [29]:
# Complie regualr expression for any matching lines. 
# More efficent that recalling the expression everytime
re_course = re.compile(r'([A-Z]{2}[0-9]{3})  (.*)([0-9]{3})(\*?) *') #r for raw data

In [33]:
# Keeping track of the number of courses
no_lines = 0

# Looping through each line of the response.
for line in resp.iter_lines():
    # Match only the lines that match the regular expression i.e. courses
    if re.course.fullmatch(line.decode('iso-8859-1')): # Needed as CAO also uses Irish language which contains fadas
        no_lines = no_lines+1
        # Orginal list in the comment
        #print(line)
        csv_version = re_course.sub(r'\1,\2,\3,\4', line.decode('iso-8859-1')) # Getting relevant parts for lines which match
        print(csv_version) # Print CSV style

# Number of courses matched
print(f'Total number of lines matched: {no_lines}.')
    

AL801,Software Design for Virtual Reality and Gaming       ,300,
AL802,Software Design in Artificial Intelligence for Clo   ,313,
AL803,Software Design for Mobile Apps and Connected Devi   ,350,
AL805,Computer Engineering for Network Infrastructure      ,321,
AL810,Quantity Surveying                                   ,328,
AL820,Mechanical and Polymer Engineering                   ,327,
AL830,General Nursing                                      451*     ,444,
AL832,Mental Health Nursing                                440*     ,431,
AL835,Pharmacology                                         ,356,
AL836,Nutrition and Health Science                         ,346,
AL837,Sports Science with Exercise Physiology              ,357,
AL838,Biotechnology                                        ,324,
AL839,Microbiology                                         ,325,
AL840,Pharmaceutical Sciences                              ,346,
AL841,Athletic and Rehabilitation Therapy                  477      ,476

## References

## End